# *Handling Missing Data with Pandas*

In [1]:
import numpy as np
import pandas as pd

# Pandas Utility Functions
Similarly to `numpy`, pandas also have a few utility functions to identify and detect null values:

In [2]:
pd.isnull(np.nan)

True

In [3]:
pd.isnull(None)

True

In [4]:
pd.isna(np.nan)

True

In [5]:
pd.isna(None)

True

The opposite ones also exist:

In [6]:
pd.notnull(None)

False

In [7]:
pd.notnull(np.nan)

False

In [8]:
pd.notnull(3) #not null of 3.

True

These functions also work with Series and `DataFrame`s:

In [9]:
pd.isnull(pd.Series([1, np.nan, 7]))

0    False
1     True
2    False
dtype: bool

In [10]:
pd.notnull(pd.Series([1, np.nan, 7]))

0     True
1    False
2     True
dtype: bool

In [11]:
pd.isnull(pd.DataFrame({
    'Column A': [1, np.nan, 7],
    'Column B': [np.nan, 2, 3],
    'Column C': [np.nan, 2, np.nan]
}))

,Column A,Column B,Column C
0,False,True,True
1,True,False,False
2,False,False,True


# Pandas Operating with Missing Values
Pandas manages missing values more gracefully then numpy. `nan`s will no longer behave as 'viruses', and operations will just ignore the completely:

In [12]:
s = pd.Series([1, 2, 3, np.nan, None, 4])

In [13]:
pd.notnull(s)

0     True
1     True
2     True
3    False
4    False
5     True
dtype: bool

In [14]:
pd.isna(s)

0    False
1    False
2    False
3     True
4     True
5    False
dtype: bool

In [15]:
pd.notnull(s).sum()

4

In [16]:
pd.isnull(s).sum()

2

In [17]:
pd.notnull(s).count()

6

In [18]:
s[pd.notnull(s)]

0    1.0
1    2.0
2    3.0
5    4.0
dtype: float64

Both `notnull` and `isnull` are also methods of `Series` and `DataFrame`s, so we could use it that way:

In [19]:
s.isnull()

0    False
1    False
2    False
3     True
4     True
5    False
dtype: bool

In [20]:
s.notna()

0     True
1     True
2     True
3    False
4    False
5     True
dtype: bool

In [21]:
s[s.notnull()]

0    1.0
1    2.0
2    3.0
5    4.0
dtype: float64

# Dropping Null Values
Boolean selection + `notnull()` seems a little bit verbose and repetitive. And as we said before: Any repetitive task will probably have a better, more DRY way. In this case, we can use `dropna` method:

In [22]:
s.dropna()

0    1.0
1    2.0
2    3.0
5    4.0
dtype: float64

# Dropping Null Values on DataFrames
You saw how simple it is to drop `na`s with a Series. But with `DataFrame`s, there will be a few more things to consider, because you can't drop single values. You can only drop entire columns or rows. Let's start with a sample `DataFrame`:

In [23]:
df = pd.DataFrame({
    'Column A': [1, np.nan, 30, None],
    'Column B': [2, 8, 31, np.nan],
    'Column C': [np.nan, 9, 32, 100],
    'Column D': [5, 8, 34, 110]
})

df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [24]:
df.isnull()

,Column A,Column B,Column C,Column D
0,False,False,True,False
1,True,False,False,False
2,False,False,False,False
3,True,True,False,False


In [25]:
df.isnull().sum()

Column A    2
Column B    1
Column C    1
Column D    0
dtype: int64

The default `dropna` behavior will drop all the rows in which any null value is present:

In [26]:
df.dropna()

,Column A,Column B,Column C,Column D
2,30.0,31.0,32.0,34


In this case, we're dropping **rows**. Rows containing null values are dropped from DF. You can also use the `axis` parameter to drop columns containing null values:

In [27]:
df.dropna(axis = 1)  #axis = 'columns' also works

,Column D
0,5
1,8
2,34
3,110


In this case, any row or column that contains **at least** one null value will be dropped. Which can be, depending on the case, too extreme. You can control this behavior with `how` parameter. Can be either `'any'` or `'all'`:

In [28]:
df2 = pd.DataFrame({
    'Column A': [1, np.nan, 30],
    'Column B': [2, np.nan, 31],
    'Column C': [np.nan, np.nan, 100]
})

df2

,Column A,Column B,Column C
0,1.0,2.0,NaN
1,NaN,NaN,NaN
2,30.0,31.0,100.0


In [29]:
df.dropna(how = 'all')

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [30]:
df.dropna(how = 'any') #default behavior

,Column A,Column B,Column C,Column D
2,30.0,31.0,32.0,34


You can also use the `thresh` parameter to indicate a *threshold* (a minimum number) of non-null values for the row/column to be kept:

In [31]:
df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [32]:
df.dropna(thresh = 3)

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34


In [33]:
df.dropna(thresh = 3, axis = 'columns')

,Column B,Column C,Column D
0,2.0,NaN,5
1,8.0,9.0,8
2,31.0,32.0,34
3,NaN,100.0,110


# Filling Null Values
Sometimes instead than dropping the null values, we might need to replace them with some other values. This highly depends on your context and the dataset you're currently working. Sometimes a `nan` can be replaced with a `0`, sometimes it can be replaced with the `mean` of the sample, and some other times you can take the closest value. Again, it depends on the context. We'll show you the different methods and mechanisms and you can then apply them to your own problem.

In [34]:
s

0    1.0
1    2.0
2    3.0
3    NaN
4    NaN
5    4.0
dtype: float64

## Filling nulls with an arbiratry value

In [35]:
s.fillna(0)

0    1.0
1    2.0
2    3.0
3    0.0
4    0.0
5    4.0
dtype: float64

In [36]:
s.fillna(s.mean())

0    1.0
1    2.0
2    3.0
3    2.5
4    2.5
5    4.0
dtype: float64

## Filling nulls with contiguous (close) values
The `method` argument is used to fill values with other values close to that null one:

In [37]:
s.fillna(method = 'ffill')

0    1.0
1    2.0
2    3.0
3    3.0
4    3.0
5    4.0
dtype: float64

In [38]:
s.fillna(method = 'bfill')

0    1.0
1    2.0
2    3.0
3    4.0
4    4.0
5    4.0
dtype: float64

This can still leave null values at the extremes of the Series/DataFrame:

In [39]:
pd.Series([np.nan, 3, np.nan, 9]).fillna(method = 'ffill')

0    NaN
1    3.0
2    3.0
3    9.0
dtype: float64

In [40]:
pd.Series([1, np.nan, 3, np.nan, np.nan]).fillna(method = 'bfill')

0    1.0
1    3.0
2    3.0
3    NaN
4    NaN
dtype: float64

## Filling null values on DataFrames
The `fillna` method also works on `DataFrame`s and it works similarly. The main differences are that you can specify the `axis` (as usual, rows or columns) to use to fill the values (specially for methods) and that you have more control on the values passed:

In [41]:
df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [42]:
df.fillna({'Column A': 0, 'Column B': 99, 'Column C': df['Column C'].mean()})

,Column A,Column B,Column C,Column D
0,1.0,2.0,47.0,5
1,0.0,8.0,9.0,8
2,30.0,31.0,32.0,34
3,0.0,99.0,100.0,110


In [43]:
df.fillna(method = 'ffill', axis = 0)

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,1.0,8.0,9.0,8
2,30.0,31.0,32.0,34
3,30.0,31.0,100.0,110


In [44]:
df.fillna(method = 'ffill', axis = 1)

,Column A,Column B,Column C,Column D
0,1.0,2.0,2.0,5.0
1,NaN,8.0,9.0,8.0
2,30.0,31.0,32.0,34.0
3,NaN,NaN,100.0,110.0


## Checking if there are NAs
The question is: Does this `Series` or `DataFrame` contain any missing value? The answer should be yes or no: `True` or `False`. How can you verify it?
**Example 1: Checking the length**
If there are missing values, `s.dropna()` will have less elements than `s`:

In [45]:
s.dropna().count()

4

In [46]:
missing_values = len(s.dropna()) != len(s)
missing_values

True

There's also a `count` method, that excludes `nan`s from its result:

In [47]:
len(s)

6

In [48]:
s.count()

4

In [49]:
missing_values = s.count() != len(s)
missing_values

True

In [50]:
df3 = pd.read_csv(
    'data/eth-price.csv', 
    header = None,
    names = ['Timestamp', 'Price'],
    index_col = 0,
    parse_dates = True
)

In [51]:
df3.head()

,Timestamp,Price
Date(UTC),UnixTimeStamp,Value
4/2/2017,1491091200,48.55
4/3/2017,1491177600,44.13
4/4/2017,1491264000,44.43
4/5/2017,1491350400,44.90


In [52]:
df3

,Timestamp,Price
Date(UTC),UnixTimeStamp,Value
4/2/2017,1491091200,48.55
4/3/2017,1491177600,44.13
4/4/2017,1491264000,44.43
4/5/2017,1491350400,44.90
...,...,...
3/28/2018,1522195200,445.93
3/29/2018,1522281600,383.90
3/30/2018,1522368000,393.82
3/31/2018,1522454400,394.07


In [53]:
df3.isnull()

,Timestamp,Price
Date(UTC),False,False
4/2/2017,False,False
4/3/2017,False,False
4/4/2017,False,False
4/5/2017,False,False
...,...,...
3/28/2018,False,False
3/29/2018,False,False
3/30/2018,False,False
3/31/2018,False,False


In [54]:
df3.isnull().sum()

Timestamp    0
Price        0
dtype: int64

## More pythonic solution `any`
The methods `any` and `all` check if either there's `any` True value in a Series or `all` the values are `True`. They work in the same way as in Python:

In [55]:
pd.Series([True, False, False]).any()

True

In [56]:
pd.Series([True, False, False]).all()

False

In [57]:
pd.Series([True, True, True]).all()

True

The `isnull()` method returned a Boolean `Series` with `True` values wherever there was a `nan`:

In [58]:
s.isnull()

0    False
1    False
2    False
3     True
4     True
5    False
dtype: bool

So we can just use the `any` method with the boolean array returned:

In [59]:
pd.Series([1, np.nan]).isnull().any()

True

In [60]:
pd.Series([1, 2]).isnull().any()

False

In [61]:
s.isnull().any()

True

In [62]:
s.isnull().all()

False

A more strict version would check onl the `values` of the Series:

In [63]:
s.isnull().values

array([False, False, False,  True,  True, False])

In [64]:
s.isnull().values.any()

True

In [65]:
s.isnull().values.all()

False

# *Cleaning Not-null Values*
After dealing with many datasets, I can tell you that 'missing data' is not a big deal. The best thing that can happen is to clearly see value like `np.nan`. The only thing you need to do is just use methods like `isnull`, `fillna`, and `dropna` and pandas will take care of the rest.

But sometimes, you can have invalid values that are not just 'missing data' (`None` or `nan`). For example:

In [66]:
df4 = pd.DataFrame({
    'Sex': ['M', 'F', 'F', 'D', '?'],
    'Age': [29, 30, 24, 290, 25]
})

df4

,Sex,Age
0,M,29
1,F,30
2,F,24
3,D,290
4,?,25


The first step to clean invalid values is to **notice** them, then **identify** them and finally handle them appropriately (remove, replace, etc). Usually, for a 'categorical' type of field (like sex, which only takes values of a discreate set `('M', 'F')`, we start by analyzing the variety of values present. For that, we use the `unique()` method:

In [67]:
df4['Sex'].unique()

array(['M', 'F', 'D', '?'], dtype=object)

In [68]:
df4['Sex'].value_counts()

F    2
D    1
M    1
?    1
Name: Sex, dtype: int64

Clearly, if you see values like `'D'` or `'?'`, it'll immediately raise your attention. Now, what to do with them? Let's say you picked up the phone, called the survey company and they told you that `'D'` was a typo and it should actually be `F`. You can use `replace` fuction to replace these values:

In [69]:
df4['Sex'].replace('D', 'F')

0    M
1    F
2    F
3    F
4    ?
Name: Sex, dtype: object

It can accept a dictionary values to replace. For example, they also told you that there might be a few `'N's`, that should actually be `'M's`:

In [70]:
df4['Sex'].replace({'D': 'F', 'N': 'M'})

0    M
1    F
2    F
3    F
4    ?
Name: Sex, dtype: object

If you have many columns to replace, you could apply it at 'DataFrame level':

In [71]:
df4.replace({
    'Sex': {
        'D': 'F',
        'N': 'M'
    },
    'Age': {
        290: 29
    }
})

,Sex,Age
0,M,29
1,F,30
2,F,24
3,F,29
4,?,25


In the previous example, I explicitly replaced 290 with 29 (assuming it was just an extra 0 entered at data-entry phase). But what if you'd like to remove all the extra 0s from the ages columns?

The first step would be to just set the limit of the 'not possible' age. Let's say that anything above 100 isn't credible for our dataset. We can then combine boolean selection with the operation:

In [72]:
df4[df4['Age'] > 100]

,Sex,Age
3,D,290


And we can now just divide by 10:

In [73]:
df4.loc[df4['Age'] > 100, 'Age'] = df4.loc[df4['Age'] > 100, 'Age'] / 10

In [74]:
df4['Sex'] = df4['Sex'].replace({'D': 'F', '?': 'M'})

In [75]:
df4

,Sex,Age
0,M,29.0
1,F,30.0
2,F,24.0
3,F,29.0
4,M,25.0


## Duplicates
Checking duplicate value is extremely simple. It'll behave differently between Series and DataFrames. Let's start with Series. As an example, let's say we're throwing a fancy party and we're inviting Ambassadors from Europe. But can only invite one ambassador per country. This is our original list, and as you can see, both the UK and Germany have duplicated ambassadors:

In [76]:
ambassadors = pd.Series([
    'France', 'UK', 'UK', 'Italy', 'Germany', 'Germany', 'Germany'
], index = ['Gerard Araud', 'Kim Darroch', 'Peter Westmacott', 'Armando Varrichio', 'Petter Wittig',
            'Petter Ammon', 'Klaus Scharioth'
])

ambassadors

Gerard Araud          France
Kim Darroch               UK
Peter Westmacott          UK
Armando Varrichio      Italy
Petter Wittig        Germany
Petter Ammon         Germany
Klaus Scharioth      Germany
dtype: object

In [77]:
ambassadors.duplicated()

Gerard Araud         False
Kim Darroch          False
Peter Westmacott      True
Armando Varrichio    False
Petter Wittig        False
Petter Ammon          True
Klaus Scharioth       True
dtype: bool

In [78]:
ambassadors.duplicated(keep = 'last')

Gerard Araud         False
Kim Darroch           True
Peter Westmacott     False
Armando Varrichio    False
Petter Wittig         True
Petter Ammon          True
Klaus Scharioth      False
dtype: bool

In [79]:
ambassadors.duplicated(keep = False)

Gerard Araud         False
Kim Darroch           True
Peter Westmacott      True
Armando Varrichio    False
Petter Wittig         True
Petter Ammon          True
Klaus Scharioth       True
dtype: bool

In [80]:
ambassadors.drop_duplicates()

Gerard Araud          France
Kim Darroch               UK
Armando Varrichio      Italy
Petter Wittig        Germany
dtype: object

In [81]:
ambassadors.drop_duplicates(keep = 'last')

Gerard Araud          France
Peter Westmacott          UK
Armando Varrichio      Italy
Klaus Scharioth      Germany
dtype: object

In [82]:
ambassadors.drop_duplicates(keep = False)

Gerard Araud         France
Armando Varrichio     Italy
dtype: object

## Duplicates in DataFrames
Conceptuall speaking, duplicates in a DataFrame happen at 'row' level. Two rows with exactly the same values are considered to be duplicates:

In [83]:
players = pd.DataFrame({
    'Name': [
        'Kobe Bryant',
        'Lebron James',
        'Kobe Bryant',
        'Carmelo Anthony',
        'Kobe Bryant'
    ],
    'Position': ['SG', 'SF', 'SG', 'SF', 'SF']
})

players

,Name,Position
0,Kobe Bryant,SG
1,Lebron James,SF
2,Kobe Bryant,SG
3,Carmelo Anthony,SF
4,Kobe Bryant,SF


In [84]:
players.duplicated()

0    False
1    False
2     True
3    False
4    False
dtype: bool

In [85]:
players.duplicated(subset = ['Name'])

0    False
1    False
2     True
3    False
4     True
dtype: bool

In [86]:
players.duplicated(subset = ['Name'], keep = 'last')

0     True
1    False
2     True
3    False
4    False
dtype: bool

In [87]:
players.drop_duplicates()

,Name,Position
0,Kobe Bryant,SG
1,Lebron James,SF
3,Carmelo Anthony,SF
4,Kobe Bryant,SF


In [88]:
players.drop_duplicates(subset = ['Name'])

,Name,Position
0,Kobe Bryant,SG
1,Lebron James,SF
3,Carmelo Anthony,SF


In [89]:
players.drop_duplicates(subset = ['Name'], keep = 'last')

,Name,Position
1,Lebron James,SF
3,Carmelo Anthony,SF
4,Kobe Bryant,SF


## Text Handling
Cleaning text values can be incredibly hard, invalid text values involves, 99% of the time, mistyping, which is completely unpredictable and doesn't follow any pattern. Thankfully, it's not so common these days, where data-entry tasks have been replaced by machines. Still, let's explore the most common cases:

## Splitting Columns
The result of a survey is loaded and this is what you get:

In [90]:
df5 = pd.DataFrame({
    'Data': [
        '1987_M_US_1',
        '1990?_M_UK_1',
        '1992_F_US_2',
        '1970?_M_   IT_1',
        '1985_F_I  T_2'
    ]
})

df5

,Data
0,1987_M_US_1
1,1990?_M_UK_1
2,1992_F_US_2
3,1970?_M_ IT_1
4,1985_F_I T_2


You know that a the single columns represent the values 'Year, Sex, Country, and Number of children'. But it's all been grouped in the same column and separated by an underscore. Pandas has a convenient method named `split` that we can use in these situations:

In [91]:
df5['Data'].str.split('_')

0        [1987, M, US, 1]
1       [1990?, M, UK, 1]
2        [1992, F, US, 2]
3    [1970?, M,    IT, 1]
4      [1985, F, I  T, 2]
Name: Data, dtype: object

In [92]:
df5['Data'].str.split('_', expand = True)

,0,1,2,3
0,1987,M,US,1
1,1990?,M,UK,1
2,1992,F,US,2
3,1970?,M,IT,1
4,1985,F,I T,2


In [93]:
df5 = df5['Data'].str.split('_', expand = True)

In [94]:
df5

,0,1,2,3
0,1987,M,US,1
1,1990?,M,UK,1
2,1992,F,US,2
3,1970?,M,IT,1
4,1985,F,I T,2


In [95]:
df5.columns = ['Year', 'Sex', 'Country', 'Children']

In [96]:
df5

,Year,Sex,Country,Children
0,1987,M,US,1
1,1990?,M,UK,1
2,1992,F,US,2
3,1970?,M,IT,1
4,1985,F,I T,2


You can also check which columns contains given a value with the `contains` method:

In [97]:
df5['Year'].str.contains('\?')

0    False
1     True
2    False
3     True
4    False
Name: Year, dtype: bool

Removing blank space like `'I  T'` can be achieved with `strip` (`lstrip` and `rstrip` also exist) or just `replace`:

In [98]:
df5['Country'].str.strip()

0      US
1      UK
2      US
3      IT
4    I  T
Name: Country, dtype: object

In [99]:
df5['Country'].str.replace('  ', '')

0     US
1     UK
2     US
3     IT
4     IT
Name: Country, dtype: object

In [100]:
df5['Year'].str.replace('?', '')

0    1987
1    1990
2    1992
3    1970
4    1985
Name: Year, dtype: object

In [101]:
df5['Year'].str.replace(r'(?P<year>\d{4})\?', lambda m: m.group('year'))

0    1987
1    1990
2    1992
3    1970
4    1985
Name: Year, dtype: object

In [102]:
df5

,Year,Sex,Country,Children
0,1987,M,US,1
1,1990?,M,UK,1
2,1992,F,US,2
3,1970?,M,IT,1
4,1985,F,I T,2


In [103]:
df5['Country'] = df5['Country'].str.replace('  ', '')

In [104]:
df5['Year'] = df5['Year'].str.replace('?', '')

In [105]:
df5

,Year,Sex,Country,Children
0,1987,M,US,1
1,1990,M,UK,1
2,1992,F,US,2
3,1970,M,IT,1
4,1985,F,IT,2
